# Simple Auto Backward Elimination for Linear Regression

Download ข้อมูล boston house price จาก sklearn

In [ ]:
!pip install statsmodels --user

In [1]:
from sklearn.datasets import load_boston

import numpy as np
import statsmodels.api as sm

data = load_boston()

print(data.data,'\n\nFeature Names:', data.feature_names)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]] 

Feature Names: ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [2]:
X = data.data
y = data.target

# สร้าง list สำหรับเก็บชื่อ feature
'''
เติมชื่อ feature 'constant' ไว้ตัวแรกของ list feat_name ก่อน เพราะตัว model OLS ของ library statsmodel ตอนทำ linear regression
จะไม่เติม constant ให้ เราต้องมาเติมเอง manually ด้วย sm.add_constant() ตามใน function ด้านล่าง
ดังนั้นถ้าจะ fit linear regression โดยให้มีค่า constant ด้วย ก็ควรจะ add ชื่อมันมาเก็บไว้ด้วย
'''
feat_name = ['constant']
feat_name += list(data.feature_names)

สร้าง funciton สำหรับทำ backward elimination ver1: ตัดตัวแปรจากค่า p-value อย่างเดียวจนกว่าทุกตัวแปรมีค่า p-value ต่ำกว่า significance level ที่ตั้งไว้

In [3]:
def backward_elim(X,y,feat_name, sl):
    
    mod_feat_name = feat_name.copy()
    numVars = X.shape[1]
    
    # เติม column ที่มีค่าเป็น 1 ทั้งหมด ไว้ข้างหน้า (สำหรับเป็นจุดตัดแกน y ใน สมการ linear)
    X = sm.add_constant(X)
    
    for i in range(numVars):
      
        regressor_OLS = sm.OLS(y, X).fit()
        p_max = np.max(regressor_OLS.pvalues)
        
        if p_max > sl:
          X = np.delete(X, np.argmax(regressor_OLS.pvalues), 1)
          del mod_feat_name[np.argmax(regressor_OLS.pvalues)]
        else:
          break
          
    print(regressor_OLS.summary())
    return X, mod_feat_name

In [4]:
X_bwe,mod_feat_name = backward_elim(X,y,feat_name,0.05)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     128.2
Date:                Fri, 19 Apr 2019   Prob (F-statistic):          5.54e-137
Time:                        11:48:18   Log-Likelihood:                -1498.9
No. Observations:                 506   AIC:                             3022.
Df Residuals:                     494   BIC:                             3072.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.3411      5.067      7.171      0.0

In [5]:
mod_feat_name

['constant',
 'CRIM',
 'ZN',
 'CHAS',
 'NOX',
 'RM',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']

สร้าง funciton สำหรับทำ backward elimination ver2: ตัดตัวแปรจากค่า p-value  และเช็คว่า adjust R sqaured ไม่ลดลง

In [6]:
def backward_elim2(X,y,feat_name, sl):
  
    mod_feat_name = feat_name.copy()
    numVars = X.shape[1]
    
    # เติม column ที่มีค่าเป็น 1 ทั้งหมด ไว้ข้างหน้า (สำหรับเป็นจุดตัดแกน y ใน สมการ linear)
    X = sm.add_constant(X)
    
    for i in range(numVars):
      
        regressor_OLS = sm.OLS(y, X).fit()
        p_max = np.max(regressor_OLS.pvalues)
        adjR_before = regressor_OLS.rsquared_adj
        
        if p_max > sl:
            
            idx_pmax = np.argmax(regressor_OLS.pvalues)
            temp = X[:,idx_pmax]
            X = np.delete(X, idx_pmax, 1)
            tmp_regressor = sm.OLS(y, X).fit()
            del_feat_name = mod_feat_name[idx_pmax]
            mod_feat_name.remove(del_feat_name)
            adjR_after = tmp_regressor.rsquared_adj
            
            # incase of rollback
            if (adjR_before >= adjR_after):
                X = np.concatenate((X,temp),axis=1)
                mod_feat_name.append(del_feat_name)
                print (regressor_OLS.summary())
                return X, mod_feat_name
            else:
                continue
                        
        else:
          break
          
    print(regressor_OLS.summary())
    return X,mod_feat_name

In [7]:
X_bwe,mod_feat_name = backward_elim2(X,y,feat_name,0.05)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     128.2
Date:                Fri, 19 Apr 2019   Prob (F-statistic):          5.54e-137
Time:                        11:48:21   Log-Likelihood:                -1498.9
No. Observations:                 506   AIC:                             3022.
Df Residuals:                     494   BIC:                             3072.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.3411      5.067      7.171      0.0

In [8]:
mod_feat_name

['constant',
 'CRIM',
 'ZN',
 'CHAS',
 'NOX',
 'RM',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']